# Racing line calculation and visualisation

The approach is to smooth central line of a given track. This is achieved by minimising the distance between each pair
 of two closes waypoints. Selected closes waypoints are not always the nearest waypoints, such as between a pair of
 selected waypoints, there might be others waypoints located closer. However, these are still close enough to be used.
 This is controlled by skipp_step parameter, which allows acceleration of the algorithm. The optimisation algorithm
 takes into account track's inner and outer borders to ensure that new calculated waypoints do not touch the borders or
 go beyond the borders or located too close to the borders. This is controlled by  max_offset parameter.

This algorithm does not search for an optimal arc of corners to race at the maximum possible speed for a given steering.

In [ ]:
import glob
import math
import os.path
import importlib.util
import numpy as np
import matplotlib.pyplot as plt
from src.reward_function import calc_distance
from src.reward_function import smooth_central_line

def load_track_definition(track_name):
    spec = importlib.util.spec_from_file_location(track_name, "../tracks/%s.py" % track_name)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module.track_width, np.asarray(module.track_original)

def calc_track_borders(track_width, center_line):
    delta_x = (np.roll(center_line[:,0], 1) - np.roll(center_line[:,0], -1))
    delta_y = (np.roll(center_line[:,1], 1) - np.roll(center_line[:,1], -1))
    delta_xy = np.vstack((delta_x, delta_y)).T
    l2_norm = np.array([math.hypot(dx, dy) for [dx, dy] in delta_xy])
    sinAlpa = delta_x / l2_norm
    cosAlpa = delta_y / l2_norm

    half_track_width = track_width * 0.5
    inner_border_x = center_line[:,0] + cosAlpa * half_track_width
    inner_border_y = center_line[:,1] - sinAlpa * half_track_width

    outer_border_x = center_line[:,0] - cosAlpa * half_track_width
    outer_border_y = center_line[:,1] + sinAlpa * half_track_width

    inner_border = np.vstack((inner_border_x, inner_border_y)).T
    outer_border = np.vstack((outer_border_x, outer_border_y)).T
    return inner_border, outer_border

track_files = glob.glob("../tracks/*.npy")
track_names = list(map(lambda x: os.path.basename(x).split(".npy")[0], track_files))
print("Available tracks:")
track_names

In [ ]:
TRACK_NAME = "Po-Chun_Speedway"
print("Track: ", TRACK_NAME)

In [ ]:
##### Use the following steps to create new .npy file for a track, if not available above.
new_track_width, new_center_line = load_track_definition(TRACK_NAME)
new_inner_border, new_outer_border = calc_track_borders(new_track_width, new_center_line)
NEW_WAYPOINTS = np.concatenate((new_center_line, new_inner_border, new_outer_border), axis=1)
with open("../tracks/%s.npy" % TRACK_NAME, "wb") as f:
    np.save(f, np.asarray(NEW_WAYPOINTS))

In [ ]:
# Load track coordinates
track_width, _ = load_track_definition(TRACK_NAME)
waypoints = np.load("../tracks/%s.npy" % TRACK_NAME)
center_line = waypoints[:,0:2]
inner_border = waypoints[:,2:4]
outer_border = waypoints[:,4:6]

def plot_track(smooth_line, inner_border, outer_border, center_line = None):
    fig, ax = plt.subplots()
    ax.set_facecolor("#103010")
    if center_line is not None:
            ax.plot(center_line[:,0], center_line[:,1], 'y--', alpha=0.3, label = 'center_line')
    ax.plot(smooth_line[:,0], smooth_line[:,1], 'y--', label = 'smooth_line')
    ax.plot(inner_border[:,0], inner_border[:,1], 'w-', alpha=0.3, label = 'inner_border')
    ax.plot(outer_border[:,0], outer_border[:,1], 'w-', alpha=0.3, label = 'outer_border')
    ax.fill_between(outer_border[:,0], outer_border[:,1], outer_border[:+1,1], facecolor="#03030f")
    ax.fill_between(inner_border[:,0], inner_border[:,1], inner_border[:+1,1], facecolor="#103010")
    fig.tight_layout()
    plt.axis('scaled')
    return plt


def show_smooth_line(smooth_cline, center_line, inner_border, outer_border):
    # These should be the same
    np_smooth_cline = np.asarray(smooth_cline)
    length_of_center_line = 0.0
    for i in range(1, len(center_line)):
        length_of_center_line += calc_distance(center_line[i - 1], center_line[i])
    length_of_smooth_cline = 0.0
    for i in range(1, len(smooth_cline)):
        length_of_smooth_cline += calc_distance(smooth_cline[i - 1], smooth_cline[i])
    for i in range(1, len(smooth_cline)):
        border_width = calc_distance(center_line[i], inner_border[i])
        racing_width = calc_distance(center_line[i], smooth_cline[i])
        if racing_width > border_width:
            print("WARNING: Racing line is off border: ", "\nborder width =",border_width, "\nracing width =", racing_width)
        break
    print("Original line length: %0.2f" % length_of_center_line)
    print("New race line length: %0.2f" % length_of_smooth_cline)
    return plot_track(np_smooth_cline, inner_border, outer_border, center_line)


track = plot_track(center_line, inner_border, outer_border)
track.savefig("../tracks/%s.png" % TRACK_NAME, bbox_inches='tight', dpi=200)
track.show()

In [ ]:
# maximum allowed offset of racing line from central line in percentage
RACING_LINE_VS_CENTRAL_LINE = 0.90
max_offset = track_width * RACING_LINE_VS_CENTRAL_LINE * 0.5

smooth_cline1 = smooth_central_line(center_line, max_offset)
track = show_smooth_line(smooth_cline1, center_line, inner_border, outer_border)
track.savefig("../tracks/%s_v1.png" % TRACK_NAME, bbox_inches='tight', dpi=200)
track.show()

In [ ]:
smooth_cline2 = smooth_central_line(center_line, max_offset, skip_step=2)
track = show_smooth_line(smooth_cline2, center_line, inner_border, outer_border)
track.savefig("../tracks/%s_v2.png" % TRACK_NAME, bbox_inches='tight', dpi=200)
track.show()

In [ ]:
smooth_cline3 = smooth_central_line(center_line, max_offset, 0.10, 0.15, 0.50, 0.15, 0.10, skip_step=3)
track = show_smooth_line(smooth_cline3, center_line, inner_border, outer_border)
track.savefig("../tracks/%s_v3.png" % TRACK_NAME, bbox_inches='tight', dpi=200)
track.show()

In [ ]:
smooth_cline2